In [7]:
import pulp as p
import pandas as pd

# Test data
# part_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= "Test Part Data")
# stock_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= "Test Stock Data")

# .049 data
# part_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= ".049 Part Data")
# stock_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= ".049 Stock Data")

# .065 data
part_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= ".065 Part Data")
stock_df = pd.read_excel("Tube Cut LP Test Data.xlsx", sheet_name= ".065 Stock Data")

prob = p.LpProblem("Tube_Cut",p.LpMinimize)

part_list = list(map(int, part_df["Part Number"]))
stock_list = list(map(int, stock_df["Stock Number"]))

part_on_stock = [(i, j) for i in part_list for j in stock_list]

part_lengths = dict(zip(part_list, part_df["Length"]))
stock_lengths = dict(zip(stock_list, stock_df["Length"]))

part_vars = p.LpVariable.dicts("Part_Stock", (part_list, stock_list), 0, 1, cat='Integer')
stock_vars = p.LpVariable.dicts("Stock",stock_list, 0, 1, cat='Integer')

prob += (p.lpSum([stock_lengths[j] * stock_vars[j] - 
                 p.lpSum([part_lengths[i] * part_vars[i][j] for i in part_list]) 
                 for j in stock_list]), 
                 "Excess Stock")
for j in stock_list:
    prob += (p.lpSum([part_lengths[i] * part_vars[i][j] for i in part_list]) - stock_lengths[j] * stock_vars[j] <= 0, 
             ("Stock Capacity " + str(j)))
             
for i in part_list:
    prob += (p.lpSum([part_vars[i][j] for j in stock_list]) - 1 == 0, 
             ("Part Cut " + str(i)))

In [8]:
print(prob)
prob.solve()
# The status of the solution is printed to the screen
print("Status:", p.LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    if v.varValue != 0:
        print(v.name, "=", v.varValue)
part_on_stock = [v.varValue for v in prob.variables()]

# The optimised objective function value is printed to the console
print("Excess Material = ", p.value(prob.objective))


Tube_Cut:
MINIMIZE
-29.77*Part_Stock_21_1 + -29.77*Part_Stock_21_2 + -29.77*Part_Stock_21_3 + -29.77*Part_Stock_21_4 + -29.77*Part_Stock_21_5 + -29.77*Part_Stock_21_6 + -29.77*Part_Stock_21_7 + -29.77*Part_Stock_21_8 + -29.77*Part_Stock_22_1 + -29.77*Part_Stock_22_2 + -29.77*Part_Stock_22_3 + -29.77*Part_Stock_22_4 + -29.77*Part_Stock_22_5 + -29.77*Part_Stock_22_6 + -29.77*Part_Stock_22_7 + -29.77*Part_Stock_22_8 + -30.25*Part_Stock_23_1 + -30.25*Part_Stock_23_2 + -30.25*Part_Stock_23_3 + -30.25*Part_Stock_23_4 + -30.25*Part_Stock_23_5 + -30.25*Part_Stock_23_6 + -30.25*Part_Stock_23_7 + -30.25*Part_Stock_23_8 + -30.25*Part_Stock_24_1 + -30.25*Part_Stock_24_2 + -30.25*Part_Stock_24_3 + -30.25*Part_Stock_24_4 + -30.25*Part_Stock_24_5 + -30.25*Part_Stock_24_6 + -30.25*Part_Stock_24_7 + -30.25*Part_Stock_24_8 + -15.3*Part_Stock_27_1 + -15.3*Part_Stock_27_2 + -15.3*Part_Stock_27_3 + -15.3*Part_Stock_27_4 + -15.3*Part_Stock_27_5 + -15.3*Part_Stock_27_6 + -15.3*Part_Stock_27_7 + -15.3*Part_St

In [9]:
# part_on_stock
# print(part_on_stock)
# print(part_vars)
# prob.variables()[0]

for j in stock_list:
    print(j)
    for i in part_list:
        if part_vars[i][j].varValue != 0:
            print(str(part_vars[i][j]) + ": " + str(part_vars[i][j].varValue))

1
2
Part_Stock_23_2: 1.0
Part_Stock_21_2: 1.0
3
Part_Stock_39_3: 1.0
Part_Stock_45_3: 1.0
Part_Stock_46_3: 1.0
Part_Stock_27_3: 1.0
Part_Stock_28_3: 1.0
4
5
Part_Stock_29_5: 1.0
Part_Stock_37_5: 1.0
Part_Stock_38_5: 1.0
6
Part_Stock_42_6: 1.0
Part_Stock_22_6: 1.0
7
Part_Stock_30_7: 1.0
Part_Stock_40_7: 1.0
Part_Stock_31_7: 1.0
Part_Stock_24_7: 1.0
8
Part_Stock_41_8: 1.0
Part_Stock_32_8: 1.0
Part_Stock_65_8: 1.0
